## Объединение датасетов, удаление дубликатов и пустых строк

In [ ]:
import pandas as pd

df0 = pd.read_excel('data.xlsx')
df1 = pd.read_excel('data_labeled.xlsx')

In [ ]:
data0 = df0['normalized_text']
data1 = df1['Универсальная заявка на доступ к АС']

In [ ]:
new_data = pd.concat([data0, data1])

In [ ]:
new_data = new_data.drop_duplicates()
new_data.shape

new_data = new_data.dropna()
new_data.shape

### Преобразование датасета в лист

In [ ]:
new_data_list = new_data.to_list()

## Нормализация данных с помощью UDpipe модели и фильтр-скриптов 

In [ ]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'

modelfile = wget.download(udpipe_url)

In [ ]:
def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken

In [ ]:
def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace \
        ('\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019', '\u0022', text)

    text = list_replace \
        ('\u2012\u2013\u2014\u2015\u203E\u0305\u00AF', '\u2003\u002D\u002D\u2003', text)

    text = list_replace('\u2010\u2011', '\u002D', text)

    text = list_replace \
            (
            '\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000',
            '\u2002', text)

    text = re.sub('\u2003\u2003', '\u2003', text)
    text = re.sub('\t\t', '\t', text)

    text = list_replace \
            (
            '\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062',
            '.', text)

    text = list_replace('\u2217', '\u002A', text)

    text = list_replace('…', '...', text)

    text = list_replace('\u2241\u224B\u2E2F\u0483', '\u223D', text)

    text = list_replace('\u00C4', 'A', text)  # латинская
    text = list_replace('\u00E4', 'a', text)
    text = list_replace('\u00CB', 'E', text)
    text = list_replace('\u00EB', 'e', text)
    text = list_replace('\u1E26', 'H', text)
    text = list_replace('\u1E27', 'h', text)
    text = list_replace('\u00CF', 'I', text)
    text = list_replace('\u00EF', 'i', text)
    text = list_replace('\u00D6', 'O', text)
    text = list_replace('\u00F6', 'o', text)
    text = list_replace('\u00DC', 'U', text)
    text = list_replace('\u00FC', 'u', text)
    text = list_replace('\u0178', 'Y', text)
    text = list_replace('\u00FF', 'y', text)
    text = list_replace('\u00DF', 's', text)
    text = list_replace('\u1E9E', 'S', text)

    currencies = list \
            (
            '\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192'
        )

    alphabet = list \
            (
            '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = ''.join(cleaned_text)

    return cleaned_text

In [ ]:
def process(pipeline, text='Строка', keep_pos=False, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [ ]:
new_data_list = list(map(str, new_data_list))

In [ ]:
%%time
from ufal.udpipe import Model, Pipeline
import os
import re
import sys

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]
    
    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...')
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    for_ret = []
    print('Processing input...')
    n = 0
    for line in text:
        res = unify_sym(line.strip())
        print(n)
        n += 1
        output = process(process_pipeline, text=res)
        for_ret.append(' '.join(output))
    return for_ret
     
text_new = tag_ud(new_data_list, modelfile='udpipe_syntagrus.model')

In [ ]:
df_imp = pd.DataFrame(text_new, columns=['Column1']) 
df_imp.to_excel('out.xlsx', index=True) # сохранение нормализованного датасета

## Замена запросов на неправильной раскладке

In [2]:
df = pd.read_excel('out.xlsx') #загружаем нормализованный датасет

In [5]:
data2 = df['Column1']
data2.shape

sents = data2.to_list()

In [ ]:
#лист с самыми используемыми eng словами из нашего датасета
norm_eng = ['NUM_TOKEN', 'PERSON_TOKEN', 'crm', 'success', 'http', 'factors', 'ac', 'PHONE_NUMBER_TOKEN', 'sd', 'factor', 'service', 'covid', 'selfserviceportal_ca_sbrf_ru:9080', 'new', 'transact', 'manager', 'PERIOD_TOKEN', 'face', 'way', 'pages', 'optinet', 'chasca', 'memoplayer', 'hr', 'extsystem=sm', 'outlook', 'sm', 'skype', 'GEO_TOKEN', 'https', 'back', 'office', 'smart', 'it', 'ip', 'pos', 'nice', 'alpha', 'sf', 'TIME_DAY_TOKEN', 'ci', 'sberbank', 'vpn', 'opticash', 'post', 'sber', 'back-office', 'cib', 'sim', 'is', 'nexus', 'qr', 'polycom', 'citrix', 'pega', 'web', 'access', 'nt', 'icpost', 'microsoft', 'URL_TOKEN', 'id', 'portal', 'srm', 'mis', 'CCY_TOKEN', 'data', 'wfm', 'argus', 'compass', 'ic', 'confluence', 'jira', 'cisco', 'ekp', 'qric', 'mass', 'diasoft', 'alm', 'sudirall_ca_sbrf_ru', 'defaultvalues', 'sledopyt', 'bpm', 'eks', 'user', 'soft', 'box', 'sql', 'ms', 'TIME_TEMPORAL_TOKEN', 'hp', 'point', 'mining', 'zoom', 'sccm', 'omega', 'iserve', 'fi', 'usb', 'sb', 'fa', 'directory', 'fac', 'sup', 'succers', 'friendface', 'hf', 'mac', 'sense', 'pay', 'business', 'atm', 'power', 'view', 'server', 'as', 'bitbucket', 'teradata', 'rpa', 'epson', 'wi', 'quik', 'olivetti', 'qlik', 'excel', 'addressbook', 'ekc', 'sucsess', 'braga', 'transakt', 'management', 'registry', 'light', 'fintech', 'cms', 'active', 'flat', 'net', 'abbyy', 'remote', 'sales', 'sacces', 'dealing', 'adobe', 'air', 'any', 'cdp', 'mbank', 'blue', 'faktors', 'sberwelcome', 'sbrf', 'api', 'test', 'process', 'bi', 'TIME_DATE_TOKEN', 'account', 'desktop', 'client', 'problem', 'bank', 'list', 'sme', 'vol', 'app', 'jenkins', 'wi-fi', 'policom', 'avaya', 'word', 'lightcab', 'vista', 'yandex', 'succses', 'trbonet', 'windows', 'cab', 'anyconnect', 'succsess', 'store', 'exam', 'transactsm', 'qlick', 'vip', 'tm', 'esrt', 'pegas', 'poly', 'cti', 'ecm', 'receiver', 'atlassian', 'susses', 'port', 'rci', 'pdf', 'suces', 'book', 'url', 'sucses', 'mart', 'lite', 'sberteam', 'dashboard', 'sberworks', 'apple', 'money', 'digital', 'click', 'connect', 'deep', 'tb', 'succeess', 'me', 'developer', 'flash', 'markets', 'doc', 'do', 'touch', 'custody', 'project', 'bac', 'engage', 'co', 'tambov', 'susser', 'visa', 'servic', 'ca', 'cloud', 'secces',  'realpresence', 'reader', 'pad', 'corp', 'report', 'mobile', 'plus', 'error', 'win', 'type', 'uparm', 'control', 'acrobat', 'key', 'security', 'ibm', 'index', 'pos-терминалы', 'extsystem=friend', 'web-сайт', 'succ', 'opti', 'cov', 'wifi', 'covid-19', 'ispost', 'momentum', 'covi', 'anti', 'ivr', 'ext', 'out', 'smartvista', 'pam', 'tran', 'succesfactors', 'exchange', 'airwatch', 'tagme', 'backoffice', 'rma',  'mrm',  'sppi',  'skyp', 'start', 'oracle', 'idea', 'dors', 'web-камера', 'polikom', 'alfa', 'swift', 'outluk', 'icon', 'tranzakt', 'docker', 'vdi', 'cpm', 'manage', 'tranzact', 'ssl',  'autopay', 'compas', 'cram', 'dom', 'visio', 'internet', 'zabbix', 'address', 'sberuser', 'optic', 'domclick', 'chrome', 'kovid', 'solution', 'erp', 'servise', 'explorer', 'paper', 'smartbox', 'sucsses', 'pki', 'warm', 'succees', 'bloomberg', 'faktor', 'skupe', 'navigator', 'arm', 'successfactor', 'activex', 'sa', 'alpha-sigma', 'fraud', 'terminal', 'exsel', 'calculation', 'etsm', 'watch', 'fact', 'operations',  'priority', 'private', 'premier', 'corp_mac', 'gui', 'pin', 'memory', 'software', 'google', 'bit', 'feed', 'developers', 'seccus', 'intellij', 'succtss', 'event', 'workspace', 'chat', 'sacsess',  'market', 'global', 'time', 'succts', 'stopcov', 'seccess', 'succcess', 'suceess',  'bucket',  'fuctor', 'fuctors', 'sacses']

#смена опечаток на eng раскладке

_eng_chars = u"~!@#$%^&qwertyuiop[]asdfghjkl;'zxcvbnm,./QWERTYUIOP{}ASDFGHJKL:\"|ZXCVBNM<>?"
_rus_chars = u"ё!\"№;%:?йцукенгшщзхъфывапролджэячсмитьбю.ЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭ/ЯЧСМИТЬБЮ,"
_trans_table = dict(zip(_eng_chars, _rus_chars))

def fix_layout(s):
    return u''.join([_trans_table.get(c, c) for c in s])


sents_cpy = list(sents)
sent_split = list()
misprint = list()

for key, sent in enumerate(sents_cpy):
    f = 0
    sent_split = sent.split()
    for key_s, s in enumerate(sent_split):
        if (s[0] in _eng_chars) and (s != "."):
            for m in misprint:
                if s == m:
                    sent_split[key_s] = fix_layout(s)
                    f = 1
                    break;
            if f != 1:
                for nword in norm_eng:
                    if s == nword:
                        f = 2;
                        break;
            if f != 1 and f != 2:
                for sent_cpy in sents_cpy:
                    if f == 3:
                        break;
                    sent_cpy_split = sent_cpy.split()
                    for sc in sent_cpy_split:
                        if (sc[0] not in _eng_chars) and sc != ".":
                            if sc == fix_layout(s):
                                misprint.append(s)
                                sent_split[key_s] = fix_layout(s)
                                f = 3
                                break;
    sents_cpy[key] = ' '.join(sent_split)


In [ ]:
sents_new = list(dict.fromkeys(sents_cpy)) # убираем дубликаты из листа

In [ ]:
df_imp = pd.DataFrame(sents_new, columns=['Column1'])
df_imp.to_excel("out.xlsx", index=False) # сохранение нормализованного и с устранением опечаток датасета

## Убираем все запросы, где меньше двух слов

In [ ]:
df_normalized = pd.read_excel('out.xlsx') #загружаем последний сохраненный нормализованный датасет со всеми изменениями выше
df_normalized = df_normalized.drop("Unnamed: 0", axis=1)

In [ ]:
def del_one_word_sent(sent):
    sent = str(sent)
    if len(sent.split()) > 1:
        return sent

In [ ]:
df_norm = df_normalized['Column1'].apply(del_one_word_sent)
df_norm = df_norm.dropna()

In [ ]:
df_norm = df_norm.drop_duplicates()

In [ ]:
df_norm.to_excel("res.xlsx") #сохранение итогового датасета для обучения Fasttext модели